<h1 style="display:inline-block">ActiveVoxel Design</h1><br/>

subdivide( get_faces( solid ), divisions )

In [1]:
# cube_vxs = [[ rx ry rz invp id demi? ] ...]

# invp is the level of division of the unitary space
#  (the inverse of the line parameter in parametric form l = point.p + c)
# r in rx, ry, rz is a multiplier 0 <= r <= invp

# we use this form to keep integer coordinates, to prevent problems with precision
# and prevent ranges of tolerance to use the equality operator 

In [3]:
import numpy as np

vxs_py = [[ 1, 0, 0, 1, 0, 0 ], [ 0, 0, 0, 1, 1, 0 ], [ 0, 0, 1, 1, 2, 0 ], [ 1, 0, 1, 1, 3, 0 ], 
          [ 0, 1, 1, 1, 4, 0 ], [ 1, 1, 1, 1, 5, 0 ], [ 0, 1, 0, 1, 6, 0 ], [ 1, 1, 0, 1, 7, 0 ]]

vxs = np.array( vxs_py )

print( vxs.transpose() )


[[1 0 0 1 0 1 0 1]
 [0 0 0 0 1 1 1 1]
 [0 0 1 1 1 1 0 0]
 [1 1 1 1 1 1 1 1]
 [0 1 2 3 4 5 6 7]
 [0 0 0 0 0 0 0 0]]


In [11]:
# TODO: useful?

# deriving vxs unique ids from their positions
# pmax is the max( invp ) 
pmax = 1

vx_names = { ( pmax + 1 ) * ( pmax + 1 ) * vx[ 0 ] + ( pmax + 1 ) * vx[ 1 ] + vx[ 2 ]: 
             ( vx[ 3 ], vx[ 0 ], vx[ 1 ], vx[ 2 ], pmax ) for vx in vxs_py }

print( vx_names )


{4: (1, 1, 0, 0, 1), 0: (1, 0, 0, 0, 1), 1: (1, 0, 0, 1, 1), 5: (1, 1, 0, 1, 1), 3: (1, 0, 1, 1, 1), 7: (1, 1, 1, 1, 1), 2: (1, 0, 1, 0, 1), 6: (1, 1, 1, 0, 1)}


In [7]:
# edges divided in metadata and data to allow us to perform calculations with the smallest tensor possible

# vx1 vx2 id parent demi?

# we keep subdivided edges in an hierarchy. Edges can only be subdivided if we subdivide the faces they define.
# Two connected faces always share an edge. If we subdiv one of this faces, but not the other, the child edges 
# will be considered demi-edges (they exist in only one face).

# The same happens if we subdiv a face in n slices and the other face in m slices, if m is not a multiple of n.
# both sets of child edges will be kept and will be demi-edges, each set belonging to just one face.

# If we divide a face in n slices and the other face in n or kn (a multiple of n), the edges will be promoted
# to real edges. Their parent structures can be deleted or demoted to demi-geometries and kept for selection
# purposes (they can never be promoted to real geometry again). The default behavior is to delete.

edges_meta_py = [[ 0, 1, 1, 0, 0 ], [ 1, 2, 2, 0,  0 ], [ 2, 3, 3, 0,  0 ], [ 3, 0, 4, 0, 0 ],  
                 [ 2, 4, 5, 0, 0 ], [ 4, 5, 6,  0, 0 ], [ 5, 3, 7, 0,  0 ], [ 4, 6, 8, 0, 0 ], 
                 [ 6, 7, 9, 0, 0 ], [ 7, 5, 10, 0, 0 ], [ 6, 1, 11, 0, 0 ], [ 0, 7, 12, 0, 0 ]]


edges_meta = np.array( edges_meta_py )

max_edge = np.max( edges_meta, 0 )[ 2 ]

print( max_edge )

print( edges_meta.transpose())


12
[[ 0  1  2  3  2  4  5  4  6  7  6  0]
 [ 1  2  3  0  4  5  3  6  7  5  1  7]
 [ 1  2  3  4  5  6  7  8  9 10 11 12]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]]


In [8]:
# dx dy dz divs1 chdiv1 div2 chdiv2 id
# ( dx, dy, dz ) = direction vector = vx2 - vx1
# divs[n] = how many times this edge is divided in relation to face n
# chdiv[n] = children related to face n were created dividing this edge chdiv[n] times

# we keep v2 - v1 in the edges to allow us to make calculations to the edges directly without 
# use the vertices (this implies that if we change the vertices edges must be updated).

# in this data structure we keep the divisions of children edges, demi or real

edt_py = [[ vxs_py[ id[ 0 ]][ 0 ], 
            vxs_py[ id[ 0 ]][ 1 ], 
            vxs_py[ id[ 0 ]][ 2 ], 1, 0, 1, 0, id[ 2 ]] 
              for id in edges_meta_py ]

edges_data = np.array( edt_py )

edt_py2 = [[ vxs_py[ id[ 1 ]][ 0 ], 
            vxs_py[ id[ 1 ]][ 1 ], 
            vxs_py[ id[ 1 ]][ 2 ], 0, 0, 0, 0, 0 ] 
              for id in edges_meta_py ]

ed2 = np.array( edt_py2 )

edges_data -= ed2
edt_py = edges_data.tolist()

print( edt_py )
print()
print( edges_data.transpose())


[[1, 0, 0, 1, 0, 1, 0, 1], [0, 0, -1, 1, 0, 1, 0, 2], [-1, 0, 0, 1, 0, 1, 0, 3], [0, 0, 1, 1, 0, 1, 0, 4], [0, -1, 0, 1, 0, 1, 0, 5], [-1, 0, 0, 1, 0, 1, 0, 6], [0, 1, 0, 1, 0, 1, 0, 7], [0, 0, 1, 1, 0, 1, 0, 8], [-1, 0, 0, 1, 0, 1, 0, 9], [0, 0, -1, 1, 0, 1, 0, 10], [0, 1, 0, 1, 0, 1, 0, 11], [0, -1, 0, 1, 0, 1, 0, 12]]

[[ 1  0 -1  0  0 -1  0  0 -1  0  0  0]
 [ 0  0  0  0 -1  0  1  0  0  0  1 -1]
 [ 0 -1  0  1  0  0  0  1  0 -1  0  0]
 [ 1  1  1  1  1  1  1  1  1  1  1  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  1  1  1  1  1  1  1  1  1  1  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  2  3  4  5  6  7  8  9 10 11 12]]


In [9]:
# edge1 edge2 id demi?
# id < 0 = reversed edge

# edges will be used first to define tris, and two tris always define a quad (even if we call it a quad,
# or a "face", the tris in the faces do not need to be coplanar). Just like edges tris, after subdivs,
# will be deleted and replaced by their children by default, but they can be kept as demi-tris for
# selection purposes

tris_py = [[   1,  2, 0, 0 ], [  3,   4, 1, 0 ], [  -3,  5,  2, 0 ], [  6,  7,  3, 0 ], 
           [  -6,  8, 4, 0 ], [  9,  10, 5, 0 ], [  -9, 11,  6, 0 ], [ -1, 12,  7, 0 ], 
           [ -12, -4, 8, 0 ], [ -7, -10, 9, 0 ], [ -11, -8, 10, 0 ], [ -5, -2, 11, 0 ]]

tris = np.array( tris_py )
max_tri = np.max( tris, 0 )[ 2 ]

print( tris.transpose())


[[  1   3  -3   6  -6   9  -9  -1 -12  -7 -11  -5]
 [  2   4   5   7   8  10  11  12  -4 -10  -8  -2]
 [  0   1   2   3   4   5   6   7   8   9  10  11]
 [  0   0   0   0   0   0   0   0   0   0   0   0]]


In [10]:
# tri1 tri2 id demi?

# faces and tris can only be subdived one single time. Demi is here for the same reasons that above.

faces_py = [[ 0, 1, 0, 0 ], [ 2, 3, 1, 0 ], [ 4, 5, 2, 0 ], [ 6, 7, 3, 0 ], [ 8, 9, 4, 0 ], [ 10, 11, 5, 0 ]]

faces = np.array( faces_py )

max_face = np.max( faces, 0 )[ 2 ]

print( faces.transpose())


[[ 0  2  4  6  8 10]
 [ 1  3  5  7  9 11]
 [ 0  1  2  3  4  5]
 [ 0  0  0  0  0  0]]


In [16]:
def subdivide( _faces, divs ):
    global max_tri, max_face, faces
    
    nnewf = divs * divs
    nnewt = nnewf * 2
    
    newf_py = [ id for id in range( max_tri + 1, max_tri + nnewt + 1 )]
    newf = np.array( newf_py )
    newf = np.reshape( newf, ( nnewf, 2 ))
    max_tri = max_tri + nnewt

    ids_py = [ id for id in range( max_face + 1, max_face + nnewf + 1 )]
    ids = np.array( ids_py )
    
    newf = np.c_[ newf, ids ]
    max_face = max_face + nnewf

    faces = np.vstack(( faces, newf ))
    
    _tris = [[ faces[ tri ][ 0 ], faces[ tri ][ 1 ]]  for tri in _faces ]
    _tris = [ item for sublist in _tris for item in sublist ]
    
    # TODO: pra criar os tris novos, tem q determinar as edges deles. Como as edges
    # ja podem existir, possivelmente é melhor continuar o algoritmo e gerar as 
    # novas edges primeiro
    
    _edges = [[ tris[ edge ][ 0 ], tris[ edge ][ 1 ]] for edge in _tris ]
    _edges = [ item for sublist in _edges for item in sublist ]
    print( _edges )

print( faces[ 0 ][ 0:1 ])
print()
subdivide( faces[ 0 ][ 0:1 ], 3 )


[0]

[1, 2, 3, 4]
